In [220]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import scipy.stats as stats
from scipy.stats import spearmanr
import ast
from scipy.stats import kstest

In [221]:
# Method used to decode filename of model into balancing method, imbalance %, model name, and model id returned in an array
def get_method_and_percentage(name):
    string = "_".join(name.split("_")[:-1])
    percentage = ""
    perc = 0
    for i in range(len(string)):
        if string[i].isdigit():
            perc = i
            percentage += string[i]
        elif perc>0:
            break
    perc+=2 if len(percentage)>0 else 0
    method = ""
    for i in range(perc, len(string)):
        if string[i].isdigit():
            model_id = string[i]
        else:
            method += string[i]
    method = "_".join(method.split("_")[:-1])
    model = name.split("_")[-1][:-4]
    return [method, percentage, model, model_id]



df = pd.read_csv("XAI_results_final_SVM.csv")
# for i in range(2,5):
#     df = pd.concat([df, pd.read_csv(f"XAI_results{i}.csv")])
pd.set_option('display.width', 1000)


name = pd.DataFrame(list(df["model"].map(lambda x:get_method_and_percentage(x))))
name.columns = ["Balancing method", "Imbalance %", "Model name", "Model ID"]
df = pd.concat([name, df[df.columns[2:]]], axis=1)
df = df.sort_values(["id", "Imbalance %", "Model ID", "Balancing method", "Model name"])
df.columns = [str(col).replace("eval_", "") for col in df.columns]
df


,Balancing method,Imbalance %,Model name,Model ID,id,words_sentence,lime_sentence,words_word,lime_word
2360,LLama,10,SVM,0,0,"['there', 'may', 'not', 'be', 'a', 'critic', '...","[-0.022934971744675903, -0.0011958517663429719...",['a' 'about' 'accidental' 'act' 'action' 'affe...,"[4.33616739313076e-05, -0.003697329228902472, ..."
240,LLama_complex,10,SVM,0,0,"['there', 'may', 'not', 'be', 'a', 'critic', '...","[-0.025988159852367843, 6.581122178263225e-05,...",['a' 'about' 'accidental' 'act' 'action' 'affe...,"[0.0014720104444310655, -0.0034905926641814796..."
1200,imbalanced,10,SVM,0,0,"['there', 'may', 'not', 'be', 'a', 'critic', '...","[-0.0025273509423874344, 0.0014477108266709824...",['a' 'about' 'accidental' 'act' 'action' 'affe...,"[-0.00017893744925643828, 0.000100026422368737..."
2120,LLama,10,SVM,1,0,"['there', 'may', 'not', 'be', 'a', 'critic', '...","[-0.019187657083878585, 0.0007502100698324174,...",['a' 'about' 'accidental' 'act' 'action' 'affe...,"[0.0004965719029304788, -0.005638696011876343,..."
840,LLama_complex,10,SVM,1,0,"['there', 'may', 'not', 'be', 'a', 'critic', '...","[-0.02249236333856587, 0.0013970486623292872, ...",['a' 'about' 'accidental' 'act' 'action' 'affe...,"[0.002134552624563309, -0.005568715394313648, ..."
...,...,...,...,...,...,...,...,...,...
1879,Summarization,50,SVM,4,39,"['post', 'chasing', 'amy', 'a', 'slew', 'of', ...","[-0.006673409508617923, -0.003302518305947108,...",['a' 'about' 'absolutely' 'actress' 'affection...,"[0.007313659886995239, -0.03943056670312839, 0..."
2839,Synonym replacement,50,SVM,4,39,"['post', 'chasing', 'amy', 'a', 'slew', 'of', ...","[-0.002011967401941093, 0.003023487946904732, ...",['a' 'about' 'absolutely' 'actress' 'affection...,"[0.001734781503550134, -0.022458994400458576, ..."
2799,Translation,50,SVM,4,39,"['post', 'chasing', 'amy', 'a', 'slew', 'of', ...","[-0.006630525803076841, -0.0011548571634200369...",['a' 'about' 'absolutely' 'actress' 'affection...,"[0.007536974074509293, -0.03649603137169283, 0..."
1679,imbalanced,50,SVM,4,39,"['post', 'chasing', 'amy', 'a', 'slew', 'of', ...","[-0.005686366598732487, -0.0012348522757685751...",['a' 'about' 'absolutely' 'actress' 'affection...,"[-0.0025714797093108065, -0.01913111031902743,..."


In [223]:
# Method used to calculate p_values from wilcoxon-signed rank test between two explanations
def get_p_values(df, lime="lime_sentence"):
    results = []
    
    # Selecting model trained on original data for comparison
    original = df[df["Balancing method"]=="original"]

    # Looping through imbalance % model id balancing method and instances
    for imbalance in np.unique(df["Imbalance %"]):
        for model_id in np.unique(df["Model ID"]):
            # Selecting model trained on imbalanced data for comparison
            df_tmp = df[(df["Imbalance %"]==imbalance)&(df["Model ID"]==model_id)&(df["Balancing method"]!="original")]
            imbalanced = df_tmp[df_tmp["Balancing method"]=="imbalanced"]
            # Filtering models based on balanced datasets
            other_methods = df_tmp[df_tmp["Balancing method"]!="imbalanced"]
            for method in np.unique(other_methods["Balancing method"]):
                for i in np.unique(other_methods.id)[:2]:
                    # Saving basic model identification
                    res = {}
                    res["Model ID"] = model_id
                    res["Imbalance %"] = imbalance
                    res["Balancing method"] = method
                    res["id"] = i
                    # Comparing xai from original model and xai from balanced model
                    df_tmp = other_methods[other_methods["Balancing method"]==method]
                    x1 = ast.literal_eval(df_tmp[df_tmp.id==i][lime].values[0])
                    x2 = ast.literal_eval(original[original.id==i][lime].values[0])
                    res["original"] = stats.wilcoxon(x1, x2)[1]
                    res["spearman_original"] = spearmanr(x1, x2)[0]
                    res["normality_original"] = kstest(np.array(x1)-np.array(x2), 'norm')[1]
                    # Comparing xai from imbalanced model and xai from balanced model if imbalanced model was not bad (acc>0.6 & f1!=0)
                    if len(imbalanced)>0:
                        x3 = ast.literal_eval(imbalanced[imbalanced.id==i][lime].values[0])
                        res["imbalanced"] = stats.wilcoxon(x1, x3)[1]
                        res["spearman_imbalanced"] = spearmanr(x1, x3)[0]
                        res["normality_imbalanced"] = kstest(np.array(x1)-np.array(x2), 'norm')[1]
                    results.append(res)
    return pd.DataFrame(results)

results_sentence = get_p_values(df, lime="lime_sentence")
results_word = get_p_values(df, lime="lime_word")
results_sentence.head()

,Model ID,Imbalance %,Balancing method,id,original,spearman_original,normality_original,imbalanced,spearman_imbalanced,normality_imbalanced
0,0,10,LLama,0,0.000045,0.502280,2.663103e-104,2.043570e-07,0.598325,2.663103e-104
1,0,10,LLama,1,0.003033,0.595077,2.554212e-68,1.076253e-10,0.350683,2.554212e-68
2,0,10,LLama_complex,0,0.000065,0.516538,1.581972e-105,8.233361e-07,0.590893,1.581972e-105
3,0,10,LLama_complex,1,0.007808,0.539125,2.120496e-68,2.962350e-11,0.246128,2.120496e-68
4,1,10,LLama,0,0.000767,0.606441,1.060680e-103,1.818237e-03,0.478171,1.060680e-103


In [225]:
# Method used to join results and metrics tables
def add_metrics(results, metrics):
    
    # Changing column types to get agreement between tables
    results['Model ID'] = results['Model ID'].astype(str)
    metrics['Model ID'] = metrics['Model ID'].astype(str)
    results['Imbalance %'] = results['Imbalance %'].astype(str)
    metrics['Imbalance %'] = metrics['Imbalance %'].astype(str)
    
    # Merging tables
    final = results.merge(metrics, on=['Model ID', 'Imbalance %', 'Balancing method'], how='left')
    
    # Removal of bad models that slipped through initial filtering due to a spelling mistake
    return final[(final["acc"]>=0.6) & (final["f1"]!=0)]


# Adding metric values 
metrics = pd.read_csv("nlpaug_SVM.csv")
final_sentence = add_metrics(results_sentence, metrics)
final_word = add_metrics(results_word, metrics)
display(final_sentence.head())
final_sentence.shape

,Model ID,Imbalance %,Balancing method,id,original,spearman_original,normality_original,imbalanced,spearman_imbalanced,normality_imbalanced,...,Unnamed: 0,Model name,acc,f1,precision,recall,mcc,auc,spearman,spearman_pval
20,0,20,LLama,0,0.000741,0.746986,9.353290e-108,4.191504e-05,0.727136,9.353290e-108,...,49,SVM,0.645,0.466165,0.939394,0.31,0.390645,0.8731,0.984807,2.288834e-07
21,0,20,LLama,1,0.001759,0.698369,3.983273e-71,2.213374e-07,0.682148,3.983273e-71,...,49,SVM,0.645,0.466165,0.939394,0.31,0.390645,0.8731,0.984807,2.288834e-07
22,0,20,LLama_complex,0,0.000717,0.735113,1.494479e-109,1.547129e-04,0.707549,1.494479e-109,...,59,SVM,0.645,0.466165,0.939394,0.31,0.390645,0.8817,0.972649,2.368935e-06
23,0,20,LLama_complex,1,0.001954,0.685160,8.378749e-71,4.122172e-09,0.662574,8.378749e-71,...,59,SVM,0.645,0.466165,0.939394,0.31,0.390645,0.8817,0.972649,2.368935e-06
24,1,20,LLama,0,0.000019,0.671294,9.252907e-108,5.499830e-07,0.692050,9.252907e-108,...,46,SVM,0.700,0.583333,0.954545,0.42,0.482805,0.8508,0.954412,1.788267e-05


(110, 21)

# Are differences between lime normally distributed?

In [229]:
print("Sentence")
df_tmp = final_sentence[["normality_original", "normality_imbalanced", "f1"]].dropna()

df_tmp = df_tmp.drop("f1", axis=1)
# Calculating descriptive statistics
display(pd.DataFrame(data={"mean": np.round(np.mean(df_tmp, axis=0), 4),
                    "median": np.round(np.median(df_tmp, axis=0), 4),
                    "rejection rate": np.round(np.mean(df_tmp<0.05, axis=0), 4) }))

print("Word")
df_tmp = final_sentence[["normality_original", "normality_imbalanced", "f1"]].dropna()

df_tmp = df_tmp.drop("f1", axis=1)
# Calculating descriptive statistics
pd.DataFrame(data={"mean": np.round(np.mean(df_tmp, axis=0), 4),
                    "median": np.round(np.median(df_tmp, axis=0), 4),
                    "rejection rate": np.round(np.mean(df_tmp<0.05, axis=0), 4) })

Sentence


,mean,median,rejection rate
normality_original,0.0,0.0,1.0
normality_imbalanced,0.0,0.0,1.0


Word


,mean,median,rejection rate
normality_original,0.0,0.0,1.0
normality_imbalanced,0.0,0.0,1.0


# Are balanced models more similar to original or imbalanced model?

### Sentence

In [213]:
# Method used to calculate descriptive statistics of p-values comparing XAI of balanced models to XAI of original and imbalanced models
def similarity_balanced_imbalanced(final, threshold = None, spearman=False):
    # Filtering models if threshold was provided
    if 1-spearman:
        df_tmp = final[["original", "imbalanced", "f1"]].dropna()
    else:
        df_tmp = final[["spearman_original", "spearman_imbalanced", "f1"]].dropna()
    if threshold != None:
        df_tmp = df_tmp[df_tmp.f1>=threshold]
    df_tmp = df_tmp.drop("f1", axis=1)
    # Calculating descriptive statistics
    res = pd.DataFrame(data={"mean": np.round(np.mean(df_tmp, axis=0), 4),
                        "median": np.round(np.median(df_tmp, axis=0), 4),
                        "rejection rate": np.round(np.mean(df_tmp<0.05, axis=0), 4) })
    if spearman:
        res=res.drop(["rejection rate"], axis=1)
    return res

similarity_balanced_imbalanced(final_sentence)

,mean,median,rejection rate
original,0.1030,0.0008,0.7364
imbalanced,0.0222,0.0000,0.9091


### Word

In [195]:
similarity_balanced_imbalanced(final_word)

,mean,median,rejection rate
original,0.4641,0.4536,0.0455
imbalanced,0.3072,0.1179,0.2636


### Spearman

In [217]:
print("Sentence")
display(similarity_balanced_imbalanced(final_sentence, spearman = True))
print("Word")
similarity_balanced_imbalanced(final_word, spearman = True)

Sentence


,mean,median
spearman_original,0.7411,0.7773
spearman_imbalanced,0.7173,0.7609


Word


,mean,median
spearman_original,0.7419,0.7838
spearman_imbalanced,0.7388,0.7493


# Which method results in models the most similar to original model?

### Sentence

In [238]:
# Method used to calculate descriptive statistics of p-values comparing XAI of balanced models to XAI of original model with regard to all balancing methods
def similarity_balancing_methods(final, threshold = None, spearman = False):
    # Filtering models if threshold was provided
    df_tmp = final[["Balancing method", "original", "f1"]].dropna() if 1-spearman else final[["Balancing method", "spearman_original", "f1"]].dropna()
    if threshold != None:
        df_tmp = df_tmp[df_tmp.f1>=threshold]
    df_tmp = df_tmp.drop("f1", axis=1)
    res = []
    # Calculating descriptive statistics
    for method in np.unique(df_tmp["Balancing method"]):
        df_ = df_tmp[df_tmp["Balancing method"]==method]["original"] if 1-spearman else df_tmp[df_tmp["Balancing method"]==method]["spearman_original"]
        res.append([method, np.round(np.mean(df_),4), np.round(np.median(df_),4), np.round(np.mean(df_<0.05),4)])
    r = pd.DataFrame(res)
    r.columns = ["Balancing method", "mean", "median", "rejection rate"]
    if spearman:
        r = r.drop("rejection rate", axis=1)
    return r


similarity_balancing_methods(final_sentence)

,Balancing method,mean,median,rejection rate
0,Contextual_word_embedding,0.1371,0.0466,0.50
1,LLama,0.0908,0.0001,0.85
2,LLama_complex,0.0751,0.0002,0.90
3,ROS,0.2123,0.1444,0.30
4,Spelling_mistake,0.2083,0.0556,0.50
5,Summarization,0.0000,0.0000,1.00
6,Synonym replacement,0.2432,0.1078,0.30
7,Translation,0.0000,0.0000,1.00
8,paraphrase,0.0000,0.0000,1.00


### Word

In [239]:
similarity_balancing_methods(final_word)

,Balancing method,mean,median,rejection rate
0,Contextual_word_embedding,0.5706,0.6581,0.00
1,LLama,0.4978,0.5277,0.05
2,LLama_complex,0.5217,0.5681,0.00
3,ROS,0.5391,0.5421,0.00
4,Spelling_mistake,0.5583,0.5789,0.00
5,Summarization,0.1970,0.1365,0.10
6,Synonym replacement,0.5549,0.5944,0.00
7,Translation,0.1153,0.0946,0.30
8,paraphrase,0.5307,0.4390,0.00


In [240]:
print("Sentence")
display(similarity_balancing_methods(final_sentence, spearman = True))
print("Word")
similarity_balancing_methods(final_word, spearman = True)

Sentence


,Balancing method,mean,median
0,Contextual_word_embedding,0.8246,0.8558
1,LLama,0.7764,0.8066
2,LLama_complex,0.7701,0.8041
3,ROS,0.8351,0.8552
4,Spelling_mistake,0.8571,0.8638
5,Summarization,0.4885,0.4701
6,Synonym replacement,0.8537,0.8737
7,Translation,0.5014,0.4882
8,paraphrase,0.6981,0.6900


Word


,Balancing method,mean,median
0,Contextual_word_embedding,0.8030,0.8168
1,LLama,0.7232,0.7481
2,LLama_complex,0.7225,0.7376
3,ROS,0.8065,0.8261
4,Spelling_mistake,0.8010,0.8059
5,Summarization,0.6156,0.5846
6,Synonym replacement,0.8181,0.8320
7,Translation,0.6335,0.6325
8,paraphrase,0.7924,0.7931


# Now what if we remove models that performed significantly worse than original model based on f1

In [198]:
# Getting f1 score of model trained on original data
f1 = metrics[(metrics["Balancing method"]=="original")&(metrics["Model name"]==df["Model name"].values[0])].f1.values[0]
print(f"F1 score of original model: {np.round(f1, 2)}")
print(f"Threshold- 80% of F1 of original model: {np.round(f1*0.8, 2)}")
# Calculating threshold
threshold = np.round(f1*0.8, 2)
metrics_temp = metrics[(metrics.acc>0.6) & (metrics.f1!=0)]
print(f"This further removes {len(metrics_temp[metrics_temp.f1>threshold])} ({np.round(len(metrics_temp[metrics_temp.f1>threshold])/len(metrics_temp)*100,1)}%) of remaining models")


F1 score of original model: 0.84
Threshold- 80% of F1 of original model: 0.67
This further removes 53 (77.9%) of remaining models


# Are balanced models more similar to original or imbalanced model?

### Sentence

In [199]:
similarity_balanced_imbalanced(final_sentence, threshold)

,mean,median,rejection rate
original,0.1258,0.0067,0.6778
imbalanced,0.0271,0.0002,0.8889


### Word

In [200]:
similarity_balanced_imbalanced(final_word, threshold)

,mean,median,rejection rate
original,0.4567,0.4488,0.0444
imbalanced,0.3693,0.1667,0.1222


### Spearman

In [218]:
print("Sentence")
display(similarity_balanced_imbalanced(final_sentence, threshold, spearman = True))
print("Word")
similarity_balanced_imbalanced(final_word, threshold, spearman = True)

Sentence


,mean,median
spearman_original,0.7619,0.8469
spearman_imbalanced,0.7358,0.7835


Word


,mean,median
spearman_original,0.7757,0.8128
spearman_imbalanced,0.7649,0.7691


# Which method results in models the most similar to original model?

### Sentence

In [201]:
similarity_balancing_methods(final_sentence, threshold)

,Balancing method,mean,median,rejection rate
0,Contextual_word_embedding,0.1371,0.0466,0.5
1,LLama,0.1813,0.0060,0.7
2,LLama_complex,0.1498,0.0012,0.8
3,ROS,0.2123,0.1444,0.3
4,Spelling_mistake,0.2083,0.0556,0.5
5,Summarization,0.0000,0.0000,1.0
6,Synonym replacement,0.2432,0.1078,0.3
7,Translation,0.0000,0.0000,1.0
8,paraphrase,0.0000,0.0000,1.0


### Word

In [202]:
similarity_balancing_methods(final_word, threshold)

,Balancing method,mean,median,rejection rate
0,Contextual_word_embedding,0.5706,0.6581,0.0
1,LLama,0.5158,0.5414,0.0
2,LLama_complex,0.5284,0.5816,0.0
3,ROS,0.5391,0.5421,0.0
4,Spelling_mistake,0.5583,0.5789,0.0
5,Summarization,0.1970,0.1365,0.1
6,Synonym replacement,0.5549,0.5944,0.0
7,Translation,0.1153,0.0946,0.3
8,paraphrase,0.5307,0.4390,0.0


### Spearman

In [241]:
print("Sentence")
display(similarity_balancing_methods(final_sentence, threshold, spearman = True))
print("Word")
similarity_balancing_methods(final_word, threshold, spearman = True)

Sentence


,Balancing method,mean,median
0,Contextual_word_embedding,0.8246,0.8558
1,LLama,0.8988,0.8984
2,LLama_complex,0.8997,0.9018
3,ROS,0.8351,0.8552
4,Spelling_mistake,0.8571,0.8638
5,Summarization,0.4885,0.4701
6,Synonym replacement,0.8537,0.8737
7,Translation,0.5014,0.4882
8,paraphrase,0.6981,0.6900


Word


,Balancing method,mean,median
0,Contextual_word_embedding,0.8030,0.8168
1,LLama,0.8545,0.8560
2,LLama_complex,0.8569,0.8563
3,ROS,0.8065,0.8261
4,Spelling_mistake,0.8010,0.8059
5,Summarization,0.6156,0.5846
6,Synonym replacement,0.8181,0.8320
7,Translation,0.6335,0.6325
8,paraphrase,0.7924,0.7931
